In [ ]:
pip install pandas pyarrow boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 3.7 MB/s eta 0:00:00


In [ ]:
import boto3

# Configuração com as credenciais AWS
session = boto3.Session(
    aws_access_key_id='---', # Removido para realizar Commit através do GitHub
    aws_secret_access_key='---', # Removido para realizar Commit através do GitHub
    region_name='us-east-1'  # Escolha sua região AWS
)

# Inicializar o cliente S3
s3_client = session.client('s3')

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("brsahan/data-science-job")
print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/brsahan/data-science-job/versions/1


In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from io import BytesIO

# Exemplo de DataFrame de dados
# data = {
#     'nome': ['Alice', 'Bob', 'Charlie'],
#     'idade': [24, 27, 22],
#     'cidade': ['São Paulo', 'Rio de Janeiro', 'Curitiba']
# }
# df = pd.DataFrame(data)

# Converter DataFrame para tabela Parquet
# table = pa.Table.from_pandas(df)



# lendo o arquivo em csv
df = pd.read_csv(path + "/data_science_job.csv")
table = pa.Table.from_pandas(df)
print(df)

      work_year                             job_title  job_category  \
0          2022  Machine Learning Engineer  in office      Analysis   
1          2020                Statistician  (Remote)         ML/AI   
2          2022           Machine Learning Engineer           ML/AI   
3          2022               Data Analyst  in office         ML/AI   
4          2020                        Statistician    Data Science   
...         ...                                   ...           ...   
4995       2020   Machine Learning Engineer  (Remote)           NaN   
4996       2021   Machine Learning Engineer  (Remote)           NaN   
4997       2022           Machine Learning Engineer        Analysis   
4998       2021                        Statistician    Data Science   
4999       2020              Data Scientist  (Remote)   Engineering   

     salary_currency  salary  salary_in_usd employee_residence  \
0                EUR  186597         136086                 US   
1              

In [ ]:
# Buffer em memória para armazenar o arquivo Parquet temporariamente
buffer = BytesIO()
pq.write_table(table, buffer)
buffer.seek(0)  # Mover o cursor para o início do buffer

# Nome do bucket e do arquivo no S3
bucket_name = 'andre-victor-uninassau'
file_path = 'pasta_destino/data_science_job.parquet'

# Cria o bucket se ele nao existir
try:
  s3_client.create_bucket(Bucket=bucket_name)
  print(f"Bucket '{bucket_name}' criado com sucesso.")
except Exception as e:
  print(f"Erro ao criar o bucket '{bucket_name}': {e}")
  pass

# Carregar o arquivo Parquet no S3
s3_client.upload_fileobj(buffer, bucket_name, file_path)
print(f"Arquivo salvo no S3 em: s3://{bucket_name}/{file_path}")


Bucket 'andre-victor-uninassau' criado com sucesso.
Arquivo salvo no S3 em: s3://andre-victor-uninassau/pasta_destino/data_science_job.parquet


In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import boto3
from io import BytesIO
import datetime
import os

# Configurações do S3
partitioned_output_path = 'data_science_job_parquet_partitioned/'  # Caminho dentro do bucket S3

# Configurar os dados de exemplo
# data = {
#     'nome': ['Alice', 'Bob', 'Charlie'],
#     'idade': [24, 27, 22],
#     'cidade': ['São Paulo', 'Rio de Janeiro', 'Curitiba'],
#     'data': [
#         datetime.datetime(2023, 11, 10),
#         datetime.datetime(2023, 11, 11),
#         datetime.datetime(2023, 11, 12)
#     ]
# }

# Carregar os dados em um DataFrame do Pandas
df = pd.read_csv(path + "/data_science_job.csv")
# df = pd.DataFrame(data)

# Extrair ano, mês e dia como colunas para particionamento
# df['ano'] = df['data'].dt.year
# df['mes'] = df['data'].dt.month
# df['dia'] = df['data'].dt.day
df['work_year'] = df['work_year']

# Converter DataFrame para uma tabela do PyArrow
table = pa.Table.from_pandas(df)

# Função para salvar a tabela particionada em S3
# def save_partitioned_to_s3(table, bucket_name, output_path, partition_cols=['ano', 'mes', 'dia']):
def save_partitioned_to_s3(table, bucket_name, output_path, partition_cols=['work_year']):
    # Cria o buffer de memória para armazenar o arquivo Parquet temporariamente


    # Salva os dados particionados em um diretório temporário local
    temp_output_path = "/tmp/dados_parquet_partitioned"
    pq.write_to_dataset(
        table,
        root_path=temp_output_path,
        partition_cols=partition_cols
    )

    # Para cada arquivo Parquet particionado, faça upload para o S3
    for root, dirs, files in os.walk(temp_output_path):
        for file in files:
            if file.endswith(".parquet"):
                local_file_path = os.path.join(root, file)
                # Calcula o caminho relativo dentro do diretório temporário
                relative_path = os.path.relpath(local_file_path, temp_output_path)
                s3_file_path = os.path.join(output_path, relative_path)

                # Carregar o arquivo Parquet para o S3
                with open(local_file_path, 'rb') as f:
                    s3_client.upload_fileobj(f, bucket_name, s3_file_path)
                print(f"Arquivo carregado no S3 em: s3://{bucket_name}/{s3_file_path}")

# Executar a função de salvamento particionado
save_partitioned_to_s3(table, bucket_name, partitioned_output_path)


Arquivo carregado no S3 em: s3://andre-victor-uninassau/data_science_job_parquet_partitioned/work_year=2020/60ccd63e877e4453bd08841c01176541-0.parquet
Arquivo carregado no S3 em: s3://andre-victor-uninassau/data_science_job_parquet_partitioned/work_year=2021/60ccd63e877e4453bd08841c01176541-0.parquet
Arquivo carregado no S3 em: s3://andre-victor-uninassau/data_science_job_parquet_partitioned/work_year=2022/60ccd63e877e4453bd08841c01176541-0.parquet
